There is a pipeline for task "clean and dirty" on Kaggle 
*   The best accuracy is 0.891
*   augmentation + resnet18 + Bagging

Main steps:

1.   importing libraries
2.   removing background of pictures
3.   augmentation
4.   transforming image to matrix(dim = 2) for BaggingClassifier
5.   defining base_estimator - PytorchModel
5.   fitting/prediction


###IMPORTING

---



In [1]:
import torch
import torchvision.transforms as transforms
import glob
import matplotlib.pyplot as plt
import numpy as np
import torchvision
import time
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import sklearn



import random
seed = 77
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True


###setup config file for Kaggle 
#####For downloading datasets/sending predictions

In [ ]:
from google.colab import files
files.upload() 

!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json
!kaggle competitions download -c platesv2


In [ ]:
!unzip plates.zip 

In [4]:
import os
!rm -rf train val sample_data test
data_root = '/content/plates'
!rm -rf /content/plates/.DS_Store
!rm -rf /content/plates/train/dirty/.DS_Store
!rm -rf /content/plates/train/cleaned/.DS_Store


#Removing background for all pictures

In [ ]:
def show_input(path, title=''):
    image = Image.open(path)
    plt.imshow(image), plt.axis("off")
    plt.title(title)
    plt.pause(0.001)

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import imageio


def remove_back(listdir):
  for j in listdir:
    image_bgr = cv2.imread(j)
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    rectangle = (10, 10, image_bgr.shape[1]-20, image_bgr.shape[0]-20)

    # Create initial mask
    mask = np.zeros(image_rgb.shape[:2], np.uint8)

    # Create temporary arrays
    bgdModel = np.zeros((1, 65), np.float64)
    fgdModel = np.zeros((1, 65), np.float64)

    # Run grabCut
    cv2.grabCut(image_rgb, mask, rectangle, bgdModel, fgdModel, 10, cv2.GC_INIT_WITH_RECT) 

    # Create mask where sure and likely backgrounds set to 0, otherwise 1
    mask_2 = np.where((mask==2) | (mask==0), 0, 1).astype('uint8')

    # Multiply image with new mask to subtract background
    image_rgb_nobg = image_rgb * mask_2[:, :, np.newaxis]
    background = image_rgb - image_rgb_nobg

    # Change all pixels in the background that are not black to white
    background[np.where((background > [0, 0, 0]).all(axis = 2))] = [255, 255, 255]

    #Add the background and the image
    out_img = background + image_rgb_nobg   
    
    imageio.imwrite(j, out_img)

train_list = glob.glob('/content/plates/train/*/*.jpg')
test_list = glob.glob('/content/plates/test/*.jpg')

lists= [train_list, test_list]

for j in lists:
  remove_back(j)

for j in train_list[:10]:
  show_input(j)



##Creating train, test directories 


In [ ]:
import os
import shutil 
from tqdm import tqdm

#Creating
train_dir = 'train'
test_dir = 'test'

class_names = ['cleaned', 'dirty']

for class_name in class_names:
  os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)


#Copying
for class_name in class_names:
  os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)

for class_name in class_names:
    source_dir = os.path.join(data_root, 'train', class_name)
    for i, file_name in enumerate(tqdm(os.listdir(source_dir))):
      dest_dir = os.path.join(train_dir, class_name)  
      shutil.copy(os.path.join(source_dir, file_name), os.path.join(dest_dir, file_name))

shutil.copytree(os.path.join(data_root, 'test'), os.path.join(test_dir, 'unknown'))  

In [ ]:
import torch.nn.functional as F
class CustomNet(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.model = torchvision.models.resnet18(pretrained=True)
    for param in self.model.parameters():
      param.requires_grad = False

    self.model.fc = torch.nn.Sequential(
    torch.nn.Linear(self.model.fc.in_features, 512),
    torch.nn.BatchNorm1d(512),
    #torch.nn.Dropout(p = 0.7),
    torch.nn.ReLU(),
    torch.nn.Linear(512, 256),
    torch.nn.BatchNorm1d(256),
    #torch.nn.Dropout(p = 0.7),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 2))

  def forward(self, x):
    x = self.model(x)
    return x


#BAGGING


In [ ]:
train_transforms = transforms.Compose([
          transforms.RandomPerspective(distortion_scale=0.2, p=0.1, interpolation=3, fill=255),
          transforms.RandomChoice([transforms.CenterCrop(180),
                                  transforms.CenterCrop(160),
                                  transforms.CenterCrop(140),
                                  transforms.CenterCrop(120),
                                  transforms.Compose([transforms.CenterCrop(280),transforms.Grayscale(3,),]),
                                  transforms.Compose([transforms.CenterCrop(200),transforms.Grayscale(3,),])]),
          transforms.Resize((224, 224)),
          transforms.ColorJitter(contrast=4),
          transforms.ToTensor(),
          transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
      ])
  
val_transforms = transforms.Compose([
          transforms.RandomPerspective(distortion_scale=0.2, p=0.1, interpolation=3, fill=255),
          transforms.RandomChoice([transforms.CenterCrop(180),
                                  transforms.CenterCrop(160),
                                  transforms.CenterCrop(140),
                                  transforms.CenterCrop(120),
                                  transforms.Compose([transforms.CenterCrop(280), transforms.Grayscale(3,),]),
                                  transforms.Compose([transforms.CenterCrop(200),transforms.Grayscale(3,),])]),
          transforms.Resize((224, 224)),
          transforms.ColorJitter(contrast=4),
          transforms.ToTensor(),
          transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
      ])


###Creating custom Dataloader class for getting original tuple + path of image

In [ ]:
class ImageFolderWithPaths(torchvision.datasets.ImageFolder):
    def __getitem__(self, index):
        original_tuple = super().__getitem__(index)
        path = self.imgs[index][0]
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path
    

##Transforming image to matrix(dim = 2) for BaggingClassifier

In [ ]:
def to_tensor(phase = 'train'):
  transforms_in_place = transforms.Compose([
          transforms.Resize((224, 224)),
          transforms.ToTensor(),
          transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
  
  if phase == 'train':
    train = []
    images=glob.glob("/content/train/*/*jpg") 
    for image in images:
      img = Image.open(image)
      img = transforms_in_place(img)
      train.append((image.split('/')[3], img))
    X = np.zeros((0,150528))
    y_train = []
    for j in train:
      y_train.append(j[0])
      X = np.concatenate((X,torch.unsqueeze(j[1][:,:,:].reshape(-1), 0)), axis = 0)
    Y = []
    for j in y_train:
      if j == 'dirty':
        Y.append(1)
      else:
        Y.append(0)
    return X, Y
  
  else:
    test = []
    images=glob.glob("/content/test/unknown/*jpg") 
    for image in images:
      img = Image.open(image)
      img = transforms_in_place(img)
      test.append(img)
    X = np.zeros((0,150528))
    for j in test:
      X = np.concatenate((X,torch.unsqueeze(j[:,:,:].reshape(-1), 0)), axis = 0)
    return X

X_train, y_train = to_tensor(phase = 'train')   
X_test = to_tensor(phase = 'test')

X_train.shape, X_test.shape  # N_samples * 3 * 244 * 244

((40, 150528), (744, 150528))

##PytorchModel - base_estimator

In [ ]:
import sklearn
class PytorchModel(sklearn.base.BaseEstimator):
  def __init__(self, net_type, net_params, optim_type, optim_params, loss_fn,
               input_shape, batch_size=20, accuracy_tol=0.02, tol_epochs=10,
               cuda=True):
    self.net_type = net_type
    self.net_params = net_params
    self.optim_type = optim_type
    self.optim_params = optim_params
    self.loss_fn = loss_fn

    self.input_shape = input_shape
    self.batch_size = batch_size
    self.accuracy_tol = accuracy_tol
    self.tol_epochs = tol_epochs
    self.cuda = cuda
    self.scheduler =  torch.optim.lr_scheduler.StepLR

  """FIT FUNC"""
  def fit(self, X, y):
    self.net = self.net_type(**self.net_params)
    trainloss = []
    trainacc = []
    if self.cuda:
      self.net = self.net.cuda()
    self.optim = self.optim_type(self.net.parameters(), **self.optim_params)
    self.schedul = self.scheduler(self.optim, step_size = 5)
    uniq_classes = [0,1]
    self.classes_ = uniq_classes
 
    
    #hack
    path = '/content/train'
    train_dataset = torchvision.datasets.ImageFolder(path, transform = train_transforms)
    train_loader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True, drop_last=False)
    #hack
    
    last_accuracies = []
    epoch = 0
    keep_training = True
    while keep_training:
      self.net.train()
      train_samples_count = 0
      true_train_samples_count = 0

      self.schedul.step()
      for batch in train_loader:
        x_data, y_data = batch[0], batch[1]
        if self.cuda:
          x_data = x_data.cuda()
          y_data = y_data.cuda()

        y_pred = self.net(x_data)
        loss = self.loss_fn(y_pred, y_data)

        self.optim.zero_grad()
        loss.backward()
        self.optim.step()

        y_pred = y_pred.argmax(dim=1, keepdim=False)
        true_classified = (y_pred == y_data).sum().item()
        true_train_samples_count += true_classified
        train_samples_count += len(x_data)

      train_accuracy = true_train_samples_count / train_samples_count
      last_accuracies.append(train_accuracy)
      print('metrics for epoch', 'loss:',loss, 'acc:',train_accuracy)
      trainloss.append(loss)
      trainacc.append(train_accuracy)
      
      if len(last_accuracies) > self.tol_epochs:
        last_accuracies.pop(0)

      if len(last_accuracies) == self.tol_epochs:
        accuracy_difference = max(last_accuracies) - min(last_accuracies)
        if accuracy_difference <= self.accuracy_tol:
          keep_training = False

  
  """PREDICT FUNC"""
  def predict_proba(self, X, y=None):

    #hack
    path = '/content/test'
    test_dataset = ImageFolderWithPaths(path, val_transforms)
    test_loader = DataLoader(test_dataset, batch_size=50,shuffle=False, drop_last=False)
    #hack
    

    self.net.eval()
    predictions = []
    im_path = []
    with torch.no_grad():
      for batch in test_loader:
        x_data, y_data, paths = batch[0], batch[1], batch[2]
        if self.cuda:
          x_data = x_data.cuda()
          y_data = y_data.cuda()
        im_path.extend(paths)
        y_pred = self.net(x_data)

        predictions.append(y_pred.detach().cpu().numpy())
    #print(im_path)
    predictions = np.concatenate(predictions)
    return predictions

  def predict(self, X, y=None):
    predictions = self.predict_proba(X, y)
    predictions = predictions.argmax(axis=1)
    return predictions

#BaggingClassifier

In [ ]:
base_model = PytorchModel(net_type=CustomNet, net_params=dict(), optim_type=torch.optim.Adam,
                          optim_params={"lr": 1e-3}, loss_fn=torch.nn.CrossEntropyLoss(),
                          input_shape=(3, 224, 224), batch_size=20, accuracy_tol=0.1,
                          tol_epochs=10, cuda=True)

from sklearn.ensemble import BaggingClassifier
meta_classifier = BaggingClassifier(base_estimator=base_model, n_estimators=50) 
meta_classifier.fit(X_train, y_train) 



#PEDICTION

In [ ]:
test_img_paths =  os.listdir('/content/test/unknown/')
test_predictions = meta_classifier.predict(X_test)

for img, pred in zip(test_img_paths, test_predictions):
   show_input(img, title=pred)


In [ ]:
import pandas as pd
submission_df = pd.DataFrame.from_dict({'id': test_img_paths, 'label': test_predictions})

submission_df['label'] = submission_df['label'].map(lambda pred: 'dirty' if (pred > 0.5) else 'cleaned')
submission_df['id'] = submission_df['id'].str.replace('/content/test/unknown/', '')
submission_df['id'] = submission_df['id'].str.replace('.jpg', '')
submission_df.set_index('id', inplace=True)
submission_df.head(n=6)

submission_df.to_csv('submission.csv')
!kaggle competitions submit platesv2 -f submission.csv -m "My submission message"